In [1]:
import swarmrl.engine.resobee as resobee
import os
infomsg = "I "

import flax.linen as nn
import numpy as np
import optax
import yaml
 
import swarmrl as srl
from swarmrl.actions.actions import Action

import matplotlib.pyplot as plt

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
Could not find espressomd. Features will not be available


## RL Configuration

In [2]:

class ActoCriticNet(nn.Module):
    """A simple dense model."""

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=128)(x)
        x = nn.relu(x)
        y = nn.Dense(features=1)(x) #Critic
        x = nn.Dense(features=4)(x) #Actor
        return x, y

# Define an exploration policy
exploration_policy = srl.exploration_policies.RandomExploration(probability=0.1)

# Define a sampling_strategy
sampling_strategy = srl.sampling_strategies.GumbelDistribution()

# Value function to use
value_function = srl.value_functions.ExpectedReturns(
    gamma=0, standardize=True
)


#Define the model
actor_critic = ActoCriticNet()
network = srl.networks.FlaxModel(
        flax_model=actor_critic,
        optimizer=optax.adam(learning_rate=1),
        input_shape=(2,),
        sampling_strategy=sampling_strategy,
        exploration_policy=exploration_policy,
    )

def scale_function(distance: float):
    """
    Scaling function for the task
    """
    return 1 - distance

# task = srl.tasks.searching.GradientSensing(
#     source=np.array([10.0, 10.0]),
#     decay_function=scale_function,
#     reward_scale_factor=100,
#     box_length=np.array([20.0, 20.]),
# )
task = srl.tasks.object_movement.ResoBeeSwarm(box_length=np.array([24.0, 24.0]))

observable=srl.observables.PositionObservable(np.array([24.0,24.0]))

# Define the loss model
loss = srl.losses.PolicyGradientLoss(value_function=value_function)

actions = {
    "TranslateLeft": np.array([-10., 0.]),
    "TranslateForward": np.array([0., 10.]),
    "TranslateRight": np.array([10., 0.]),
    "MoveBackwards": np.array([0., -10.]),
}
protocol=srl.agents.ActorCriticAgent(particle_type=0, network=network, task=task, observable=observable,actions=actions,loss=loss
                                     )

# Define the force model.
rl_trainer=srl.trainers.EpisodicTrainer([protocol])


In [3]:
resobee_root_path = "/tikhome/mgern/Desktop/ResoBee/"

build_path = os.path.join(resobee_root_path, "build")
config_dir = os.path.join(resobee_root_path, 'workflow/projects/debug/parameter-combination-0/seed-0')

target = 'many_body_simulation'
resobee_executable = os.path.join(resobee_root_path, 'build/src', target)


In [4]:
system_runner = resobee.ResoBee(
    resobee_executable=resobee_executable,
    config_dir=config_dir
)

Binding to tcp address:  tcp://127.0.0.1:5544


In [5]:

with open(os.path.join(config_dir,"config.yaml"), 'r') as file:
    data = yaml.safe_load(file)
episode_length=(data["total_time"]+data["burn_in_time"])/data["integration_time_step"]

reward,force_fn=rl_trainer.perform_rl_training(
            system_runner=system_runner,
            n_episodes=100,
            episode_length=episode_length,
        )
plt.plot(reward)
plt.show()
rl_trainer.export_models("without_alignment")

Output()

/tikhome/mgern/conda/envs/manybodyrc/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:2550: UserWarning: 
Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. 
To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment 
variable. See https://github.com/google/jax#current-gotchas for more.
  start = asarray(start, dtype=computation_dtype)

/tikhome/mgern/conda/envs/manybodyrc/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:2551: UserWarning: 
Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. 
To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment 
variable. See https://github.com/google/jax#current-gotchas for more.
  stop = asarray(stop, dtype=computation_dtype)

/tikhome/mgern/conda/envs/manybodyrc/lib/python3.10/site-packages/jax/_src/numpy/array_methods.py:64: UserWarning: 
Explicitly requested dtype float64 requested in astype is not available, and will be truncated to dtype float32. To
enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. 
See https://github.com/google/jax#current-gotchas for more.
  return lax_numpy.astype(arr, dtype)

ValueError: Incompatible shapes for broadcasting: shapes=[(56, 10, 2), (56, 10)]

In [ ]:
system_runner.integrate(episode_length,force_fn)